In [ ]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 48.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f8b55b2658023c4d4e0137d9f7e0fbacbd10dea72629c341c6544d14c87ff5af
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
#  Create SparkSession object 'spark'
spark = SparkSession.builder.appName('Advertising').getOrCreate()

Linear regression without cross-valiation

Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ad = spark.read.csv('/content/drive/MyDrive/0.MKCE/Datasets/Advertising.csv', header=True, inferSchema=True)
ad.show(5)

+---+-----+-----+---------+-----+
|_c0|   TV|radio|newspaper|sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
|  4|151.5| 41.3|     58.5| 18.5|
|  5|180.8| 10.8|     58.4| 12.9|
+---+-----+-----+---------+-----+
only showing top 5 rows



Transform data structure

In [ ]:
from pyspark.ml.linalg import Vectors
ad_df = ad.rdd.map(lambda x: [Vectors.dense(x[0:3]), x[-1]]).toDF(['features', 'label'])
ad_df.show(5)

+----------------+-----+
|        features|label|
+----------------+-----+
|[1.0,230.1,37.8]| 22.1|
| [2.0,44.5,39.3]| 10.4|
| [3.0,17.2,45.9]|  9.3|
|[4.0,151.5,41.3]| 18.5|
|[5.0,180.8,10.8]| 12.9|
+----------------+-----+
only showing top 5 rows



Build linear regression model

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol = 'label')

In [ ]:
lr_model = lr.fit(ad_df)

Prediction

In [ ]:
pred = lr_model.transform(ad_df)
pred.show(5)

+----------------+-----+------------------+
|        features|label|        prediction|
+----------------+-----+------------------+
|[1.0,230.1,37.8]| 22.1| 20.60502920567592|
| [2.0,44.5,39.3]| 10.4|12.392493270268307|
| [3.0,17.2,45.9]|  9.3|12.381882445789143|
|[4.0,151.5,41.3]| 18.5|  17.6636423673972|
|[5.0,180.8,10.8]| 12.9|13.277141826096058|
+----------------+-----+------------------+
only showing top 5 rows

